In [23]:
# IE 5318 KPD Project 
# Team Red Raider Engineer
# Matthew Harvey. Willaim DuBose, Wilbert Nicolas

In [33]:
import json
import networkx as nx
import matplotlib.pyplot as plt

In [34]:
# Read data and draw some figures
import json

# Open and read the JSON file
with open('Red_Raider_Engineers.json', 'r') as file:
    data = json.load(file)

In [35]:
# Create a graph
G = nx.DiGraph()

# Create nodes with attributes
counter = 0
for element in data:
    G.add_nodes_from([(counter, element)])
    counter += 1

# Creating the donor-recipient compatiblity for matching pairs
donor_recipient_compatibility = {'O': ['O', 'A', 'B', 'AB'], 'A': ['A', 'AB'], 'B':['B', 'AB'], 'AB':['AB']}


# Create edges (relations)
for node in G.nodes:
    donors = G.nodes[node]['Donor']
    for donor in donors:
        for vertex in G.nodes:
            if node == vertex: continue
            recipient = G.nodes[vertex]['Recipient']
            if recipient not in donor_recipient_compatibility[donor]: continue
            G.add_edge(node, vertex)

In [36]:
# Finding cycles of length 3
potential_cycle_3 = []

H = G.to_undirected()

for (u,v) in H.edges:
    for k in nx.common_neighbors(H, u, v):
        potential_cycle_3.append((u,v,k))
        
#print("potential_cycle_3:", potential_cycle_3)        
        
cycle_3 = [] 

for (u,v,k) in potential_cycle_3:
    if (u,v) in G.edges and (v,k) in G.edges and (k,u) in G.edges:
        cycle_3.append((u,v,k,u))
    if (v,u) in G.edges and (u,k) in G.edges and (k,v) in G.edges: 
        cycle_3.append((v,u,k,v))

print("cycle_3:", cycle_3)

cycle_3: []


In [37]:
# Finding cycles of length 2
cycle_2 = []

for (u,v) in H.edges:
    if (u,v) in G.edges and (v,u) in G.edges:
        cycle_2.append((u,v,u))
            
print("cycles of length 2:", cycle_2)

cycles of length 2: [(0, 12, 0), (0, 18, 0), (0, 21, 0), (0, 23, 0), (0, 31, 0), (0, 33, 0), (0, 34, 0), (0, 41, 0), (0, 45, 0), (0, 46, 0), (0, 54, 0), (0, 56, 0), (0, 61, 0), (0, 66, 0), (0, 69, 0), (0, 74, 0), (0, 78, 0), (0, 81, 0), (0, 93, 0), (0, 94, 0), (0, 95, 0), (0, 100, 0), (0, 104, 0), (0, 105, 0), (0, 113, 0), (0, 116, 0), (0, 119, 0), (0, 122, 0), (0, 124, 0), (0, 127, 0), (0, 128, 0), (0, 131, 0), (0, 132, 0), (0, 140, 0), (0, 143, 0), (0, 144, 0), (0, 150, 0), (0, 152, 0), (0, 154, 0), (0, 155, 0), (0, 156, 0), (0, 160, 0), (0, 173, 0), (0, 176, 0), (0, 177, 0), (0, 179, 0), (0, 181, 0), (0, 192, 0), (0, 194, 0), (0, 196, 0), (0, 197, 0), (0, 198, 0), (0, 201, 0), (0, 204, 0), (0, 209, 0), (0, 214, 0), (0, 226, 0), (0, 228, 0), (0, 232, 0), (0, 233, 0), (0, 237, 0), (0, 239, 0), (0, 245, 0), (0, 250, 0), (0, 251, 0), (0, 252, 0), (0, 253, 0), (0, 257, 0), (0, 258, 0), (0, 259, 0), (0, 261, 0), (0, 263, 0), (0, 274, 0), (0, 278, 0), (0, 279, 0), (0, 288, 0), (0, 291, 0),

In [38]:
import gurobipy as gp
from gurobipy import GRB

# Create model object
m = gp.Model()

# Create variable for each edge
x = m.addVars(G.edges, vtype=GRB.BINARY) 
# x[u,v] = 0, if donor u is not compatible with recipient v
# x[u,v] = 1, if donor u is compatible with recipient v

# Objective function: maximize number of edges
m.setObjective(gp.quicksum(x[u,v] for u,v in G.edges()), GRB.MAXIMIZE)

# Constraint: Each donor-recipient pair is matched with one other donor-recipient pair.
m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) <= 1 for v in G.nodes)
m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) == gp.quicksum(x[(v,u)] for u in G.neighbors(v) if (v,u) in G.edges) for v in G.nodes)

# Constraint: Each donor-recipient pair is included in at most one transplantation
m.addConstrs((x[(u, v)] + gp.quicksum(x[(v, k)] for k in G.neighbors(v) if k != u) <= 1) for u, v in G.edges)

# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 193172 rows, 191680 columns and 37426436 nonzeros
Model fingerprint: 0x8bb46d8e
Variable types: 0 continuous, 191680 integer (191680 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 43156 rows and 116889 columns (presolve time = 6s) ...
Presolve removed 117607 rows and 116889 columns (presolve time = 10s) ...
Presolve removed 117607 rows and 116889 columns (presolve time = 15s) ...
Presolve removed 117607 rows and 116889 columns (presolve time = 20s) ...
Presolve removed 118768 rows and 117276 columns (presolve time = 25s) ...
Presolve removed 11

In [39]:
print("The Objective Value:",m.objVal)

# Retrieve each half of each pair to ensure matching
selected_edges = [(u,v) for (u,v) in G.edges if x[u,v].X > 0.5]
print(*selected_edges,sep='\n')

The Objective Value: 356.0
(0, 745)
(3, 484)
(7, 143)
(8, 658)
(12, 640)
(18, 630)
(20, 587)
(21, 238)
(23, 669)
(25, 56)
(30, 399)
(31, 174)
(32, 648)
(33, 442)
(34, 180)
(40, 696)
(42, 354)
(44, 739)
(45, 125)
(46, 695)
(48, 457)
(50, 627)
(52, 571)
(54, 468)
(56, 25)
(58, 596)
(61, 609)
(63, 726)
(65, 449)
(66, 430)
(67, 245)
(69, 489)
(70, 292)
(77, 558)
(78, 268)
(81, 665)
(83, 592)
(90, 522)
(93, 547)
(94, 114)
(95, 523)
(102, 591)
(104, 667)
(105, 453)
(109, 380)
(111, 703)
(113, 485)
(114, 94)
(116, 433)
(118, 352)
(119, 551)
(122, 662)
(124, 161)
(125, 45)
(127, 221)
(128, 158)
(131, 550)
(132, 668)
(133, 404)
(140, 552)
(142, 554)
(143, 7)
(144, 451)
(150, 544)
(152, 561)
(154, 686)
(155, 377)
(156, 379)
(158, 128)
(161, 124)
(164, 331)
(169, 301)
(173, 612)
(174, 31)
(176, 525)
(177, 700)
(178, 641)
(179, 691)
(180, 34)
(181, 483)
(182, 568)
(183, 743)
(188, 539)
(190, 744)
(192, 504)
(194, 254)
(196, 564)
(197, 666)
(198, 422)
(203, 646)
(204, 560)
(206, 548)
(208, 594)
(20

In [40]:
# Combine each half pair into a full cycle
selected_edges1 = []
for (u,v) in H.edges:
    if (u,v) and (v,u) in selected_edges:
       selected_edges1.append((u,v))
        
# Print the match values
print(f"Total Matched Pairs: {len(matches)}")
print(*selected_edges1,sep='\n')

Total Matched Pairs: 178
(0, 745)
(3, 484)
(7, 143)
(8, 658)
(12, 640)
(18, 630)
(20, 587)
(21, 238)
(23, 669)
(25, 56)
(30, 399)
(31, 174)
(32, 648)
(33, 442)
(34, 180)
(40, 696)
(42, 354)
(44, 739)
(45, 125)
(46, 695)
(48, 457)
(50, 627)
(52, 571)
(54, 468)
(58, 596)
(61, 609)
(63, 726)
(65, 449)
(66, 430)
(67, 245)
(69, 489)
(70, 292)
(77, 558)
(78, 268)
(81, 665)
(83, 592)
(90, 522)
(93, 547)
(94, 114)
(95, 523)
(102, 591)
(104, 667)
(105, 453)
(109, 380)
(111, 703)
(113, 485)
(116, 433)
(118, 352)
(119, 551)
(122, 662)
(124, 161)
(127, 221)
(128, 158)
(131, 550)
(132, 668)
(133, 404)
(140, 552)
(142, 554)
(144, 451)
(150, 544)
(152, 561)
(154, 686)
(155, 377)
(156, 379)
(164, 331)
(169, 301)
(173, 612)
(176, 525)
(177, 700)
(178, 641)
(179, 691)
(181, 483)
(182, 568)
(183, 743)
(188, 539)
(190, 744)
(192, 504)
(194, 254)
(196, 564)
(197, 666)
(198, 422)
(203, 646)
(204, 560)
(206, 548)
(208, 594)
(209, 427)
(210, 559)
(211, 372)
(214, 519)
(215, 387)
(219, 565)
(226, 429)
(228, 65

In [41]:
# Call in the Blood-Type values for each pair
matches = []
for u, dr1 in enumerate(data):
    for v, dr2 in enumerate(data):
        if (u, v) in selected_edges1:
            matches.append((json.dumps(dr1), json.dumps(dr2)))

# Print the final match values
print(f"The Total Combined Matched Pairs: {len(matches)}")
print(*matches,sep="\n")

The Total Combined Matched Pairs: 178
('{"Recipient": "B", "Donor": ["A"]}', '{"Recipient": "A", "Donor": ["AB", "B"]}')
('{"Recipient": "B", "Donor": ["AB", "A"]}', '{"Recipient": "A", "Donor": ["AB", "B"]}')
('{"Recipient": "B", "Donor": ["A"]}', '{"Recipient": "A", "Donor": ["AB", "B"]}')
('{"Recipient": "B", "Donor": ["A"]}', '{"Recipient": "A", "Donor": ["AB", "B"]}')
('{"Recipient": "A", "Donor": ["AB", "B"]}', '{"Recipient": "B", "Donor": ["AB", "A"]}')
('{"Recipient": "A", "Donor": ["B"]}', '{"Recipient": "B", "Donor": ["AB", "A"]}')
('{"Recipient": "B", "Donor": ["AB", "A"]}', '{"Recipient": "A", "Donor": ["AB", "B"]}')
('{"Recipient": "A", "Donor": ["AB", "B"]}', '{"Recipient": "B", "Donor": ["A"]}')
('{"Recipient": "A", "Donor": ["AB", "B"]}', '{"Recipient": "B", "Donor": ["AB", "A"]}')
('{"Recipient": "B", "Donor": ["A"]}', '{"Recipient": "A", "Donor": ["B"]}')
('{"Recipient": "B", "Donor": ["AB", "A"]}', '{"Recipient": "A", "Donor": ["AB", "B"]}')
('{"Recipient": "A", "Don